In [1]:
import pandas as pd
import boto3
import json
import configparser
from time import sleep

## Load Config Parameters
The file `dwh.cfg` contains all parameers necessary to proceed with the Cluster creation.
In addition to it, the AWS IAM Roles parameters are also defined in the file

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


## Create Clients

In [3]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       ) 

In [5]:
from botocore.exceptions import ClientError
try:
    print("Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)    
print("Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
print(roleArn)

Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhRole already exists.
Attaching Policy
Get the IAM role ARN
arn:aws:iam::500349149336:role/dwhRole


## Creating a Redshift Cluster

In [6]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)
    
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

cluster_status = 'Undefined'
while cluster_status != 'available':
    sleep(30)
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
    cluster_status = myClusterProps['ClusterStatus']
    

prettyRedshiftProps(myClusterProps)

DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

<ipython-input-6-14e4911dbc2e>:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


DWH_ENDPOINT ::  dwhcluster.cbynmx43gg9v.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::500349149336:role/dwhRole
ec2.SecurityGroup(id='sg-00282197ebcc2d058')
An error occurred (UnauthorizedOperation) when calling the AuthorizeSecurityGroupIngress operation: You are not authorized to perform this operation. Encoded authorization failure message: 1Uise4m1E83p-Plq1V8Ia54kasS48eXOxn2Rxwvq_yXb2U9SyVVrfx1-H1d8KZTbR0ShtDiFgSvJNWtIsPScgrDld6K24OTRM0ln2iaMkTQXlQ7UBiMXI8eknxIAa-EtmAKYDyHcmO1wrVVnCm5rfX2afCNJ8FFfAoN4OzWnTPko1Jz19Ov1894E6Tmg99cGEzCu0H0rp_YaP-R0Cg2a8IoCtEBeAQIgY0xz-3CJsxXt_Qhje8sNMW1Ozi6aFdqsfaQ46foWym3wXCIu08LL3IT9_sAeFGPBYpLv-gld86it0PmcFJJ959Rz0rwXYzUVKbfn3DJT3RMegaW3YTiX3Mui8yzDjxNsiBQHARNOD-Ddt3IAxCLA7oMOrKOJMzNHzNnsfu9TnftyjeDLRBIsHNJdw_Da2B_nwWl1k80j4yR2oK57wjkCaqwciYCAOAuCmj8CDbaLS95OQDvN8Wd-qmvA2xwmNeGOOPeORA7BF1gvw8_nzjomsMQ9p-bmeUc9EEG5oesRuNtL2dVtaVUu7TLi0sD3vxURwEqgBSAiSInaHOzu3WmVvp7rsdIQa9TuedRtj1RDEAwHWQVitE_xKg


In [7]:
print("host={} dbname={} user={} password={} port={}".format(DWH_ENDPOINT, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT))

host=dwhcluster.cbynmx43gg9v.us-west-2.redshift.amazonaws.com dbname=dwh user=dwhuser password=Passw0rd port=5439


## Delete Cluster

In [10]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

NameError: name 'redshift' is not defined

In [9]:
a = """
    CREATE TABLE IF NOT EXISTS staging_events (
                 num_songs           INTEGER         NULL,
                artist_id           VARCHAR         NOT NULL SORTKEY DISTKEY,
                artist_latitude     VARCHAR         NULL,
                artist_longitude    VARCHAR         NULL,
                artist_location     VARCHAR(500)   NULL,
                artist_name         VARCHAR(500)   NULL,
                song_id             VARCHAR         NOT NULL,
                title               VARCHAR(500)   NULL,
                duration            DECIMAL(9)      NULL,
                year                INTEGER         NULL
    );
""".replace("  ", " ").replace("\t","")

print(a)


  CREATE TABLE IF NOT EXISTS staging_events (
         num_songs      INTEGER     NULL,
        artist_id      VARCHAR     NOT NULL SORTKEY DISTKEY,
        artist_latitude   VARCHAR     NULL,
        artist_longitude  VARCHAR     NULL,
        artist_location   VARCHAR(500)  NULL,
        artist_name     VARCHAR(500)  NULL,
        song_id       VARCHAR     NOT NULL,
        title        VARCHAR(500)  NULL,
        duration      DECIMAL(9)   NULL,
        year        INTEGER     NULL
  );

